# Final Project - Machine Problem

### Automated Essay Scoring System
#### Miguel Christian G. Marasigan

In [24]:
#Importing libraries

from gensim.models import Word2Vec

import numpy as np
import pandas as pd
import os
import nltk
from itertools import chain
#nltk.download('stopwords') downloaded stopwords
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
import re

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

#nltk.download('punkt') downloaded punkt


In [2]:
#Importing datasets
for dirname, _, filenames in os.walk('C:\\Users\\akaii\\Downloads\\asap-aes'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\akaii\Downloads\asap-aes\.DS_Store
C:\Users\akaii\Downloads\asap-aes\Essay_Set_Descriptions.zip
C:\Users\akaii\Downloads\asap-aes\test_set.tsv
C:\Users\akaii\Downloads\asap-aes\Training_Materials.zip
C:\Users\akaii\Downloads\asap-aes\training_set_rel3.tsv
C:\Users\akaii\Downloads\asap-aes\training_set_rel3.xls
C:\Users\akaii\Downloads\asap-aes\training_set_rel3.xlsx
C:\Users\akaii\Downloads\asap-aes\valid_sample_submission_1_column.csv
C:\Users\akaii\Downloads\asap-aes\valid_sample_submission_1_column_no_header.csv
C:\Users\akaii\Downloads\asap-aes\valid_sample_submission_2_column.csv
C:\Users\akaii\Downloads\asap-aes\valid_sample_submission_5_column.csv
C:\Users\akaii\Downloads\asap-aes\valid_set.tsv
C:\Users\akaii\Downloads\asap-aes\valid_set.xls
C:\Users\akaii\Downloads\asap-aes\valid_set.xlsx


In [3]:
#dropping columns to see domain score

df = pd.read_csv('C:\\Users\\akaii\\Downloads\\asap-aes\\training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
df = df.drop(columns=['essay_id', 'essay_set'])
df = df.drop(columns=['rater1_domain1', 'rater2_domain1'])


df = df.dropna(axis=1)
df.head()



,essay,domain1_score
0,"Dear local newspaper, I think effects computer...",8
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,"Dear @LOCATION1, I know having computers has a...",8


In [4]:
lst1 = list()
lst2 = list()
lst3 = list()
lst4 = list()
lst5 = list()
lst6 = list()
lst7 = list()
lst8 = list()

In [5]:
#get essays according to their domain score
data = []
for i, j in zip(range(len(df)),df["essay"]):
    if df["domain1_score"][i] == 1:
                lst1.append(j)
    elif df["domain1_score"][i] == 2:
                lst2.append(j)
    elif df["domain1_score"][i] == 3:
                lst3.append(j)
    elif df["domain1_score"][i] == 4:
                lst4.append(j)
    elif df["domain1_score"][i] == 5:
                lst5.append(j)
    elif df["domain1_score"][i] == 6:
                lst6.append(j)
    elif df["domain1_score"][i] == 7:
                lst7.append(j)
    elif df["domain1_score"][i] == 8:
                lst8.append(j)

In [6]:
essays = list(chain(lst1,lst2,lst3,lst4,lst5,lst6,lst7,lst8))

In [7]:
def getScore(num1,num2):
    num3 = [num1]*num2
    return num3

scr1 = getScore(1,len(lst1))
scr2 = getScore(2,len(lst2))
scr3 = getScore(3,len(lst3))
scr4 = getScore(4,len(lst4))
scr5 = getScore(5,len(lst5))
scr6 = getScore(6,len(lst6))
scr7 = getScore(7,len(lst7))
scr8 = getScore(8,len(lst8))

scrTotal = list(chain(scr1,scr2,scr3,scr4,scr5,scr6,scr7,scr8))

In [8]:
#showing updated arranged dataframe
dct = {'essay': essays, 'score': scrTotal}    

df = pd.DataFrame(dct) 

df

,essay,score
0,Write a persuasive essay to a newspaper reflec...,1
1,If the people that are publishing and writing ...,1
2,I do think that there should be a censorship i...,1
3,I belive that they need to keep every thing an...,1
4,Hi my name is @CAPS1 @CAPS2 @PERSON1 and I'm h...,1
...,...,...
9563,Without patience a person can be very easily b...,8
9564,I was patient when my birthday came. I am wait...,8
9565,I was patient last year when I was when I was ...,8
9566,When I was and the @NUM1 grade and I had to pu...,8


In [9]:
swords = set(stopwords.words('english'))
snw = nltk.stem.SnowballStemmer('english')

In [11]:
text = []
for i in range(0, len(df)):
    rvw = re.sub('[^a-zA-Z]', ' ', df['essay'][i])
    rvw = rvw.lower()
    rvw = rvw.split()
    
    rvw = [snw.stem(word) for word in rvw if not word in stopwords.words('english')]
    rvw = ' '.join(rvw)
    text.append(rvw)

In [17]:
def essayToWord(essay, remove_stopwords):
    essay = re.sub("[^a-zA-Z]", " ", essay)
    words = essay.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essayToSentence(essay_v, remove_stopwords):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essayToWord(raw_sentence, remove_stopwords))
    return sentences

In [18]:
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            modelNum = np.resize(model[word],featureVec.shape)
            featureVec = np.add(featureVec, modelNum)       
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [19]:

def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy','mae'])
    model.summary()
    return model

In [28]:
X=df
y = X['score']

cv = KFold(n_splits = 5, shuffle = True)
results = []
y_pred_list = []

count = 1
for traincv, testcv in cv.split(X):
    print("\n Fold Count: " + str(count) + "\n")
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    for essay in train_essays:
            sentences += essayToSentence(essay, remove_stopwords = True)
            
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training the Model. Please Wait")
    model = Word2Vec(sentences, workers=num_workers, min_count = min_word_count, window = context, sample = downsampling)

    model.init_sims(replace=True)
    model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

    clean_train_essays = []
    
    for essay_v in train_essays:
        clean_train_essays.append(essayToWord(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essayToWord( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)
    
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50)
    y_pred = lstm_model.predict(testDataVecs)
    
    y_pred = np.around(y_pred)
    
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test.values,y_pred)
    print("\n The accuracy Score is: " + str(acc) + "\n")
    print("\n The Kappa Score is: " + str(result) + "\n")
    results.append(result)
    count += 1



 Fold Count: 1

Training the Model. Please Wait


C:\Users\akaii\AppData\Local\Temp\ipykernel_7240\3343523460.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  modelNum = np.resize(model[word],featureVec.shape)


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_24 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_25 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 5s 20ms/step - loss: 2.6871 - accuracy: 0.1662 - mae: 1.2008
Epoch 2/50
120/120 [==============================] - 2s 18ms/st

C:\Users\akaii\AppData\Local\Temp\ipykernel_7240\3343523460.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  modelNum = np.resize(model[word],featureVec.shape)


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_26 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_27 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_13 (Dropout)        (None, 64)                0         
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 5s 16ms/step - loss: 2.7146 - accuracy: 0.1666 - mae: 1.2172
Epoch 2/50
120/120 [==============================] - 2s 16ms/st

...

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_28 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_29 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_14 (Dropout)        (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 6s 18ms/step - loss: 2.6335 - accuracy: 0.1697 - mae: 1.2036
Epoch 2/50
120/120 [==============================] - 2s 19ms/st

C:\Users\akaii\AppData\Local\Temp\ipykernel_7240\3343523460.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  modelNum = np.resize(model[word],featureVec.shape)


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_31 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 6s 18ms/step - loss: 2.6970 - accuracy: 0.1707 - mae: 1.2029
Epoch 2/50
120/120 [==============================] - 2s 18ms/st

C:\Users\akaii\AppData\Local\Temp\ipykernel_7240\3343523460.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  modelNum = np.resize(model[word],featureVec.shape)


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_32 (LSTM)              (None, 1, 300)            721200    
                                                                 
 lstm_33 (LSTM)              (None, 64)                93440     
                                                                 
 dropout_16 (Dropout)        (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
120/120 [==============================] - 5s 17ms/step - loss: 2.6917 - accuracy: 0.1700 - mae: 1.2086
Epoch 2/50
120/120 [==============================] - 2s 17ms/st